In [3]:
import pandas as pd

# CONFIGURACIÓN
archivo_general = 'imco_general.csv'
archivo_desagregado = 'imco_desagregado.csv'

# ==========================================
# 1. PROCESAMIENTO: IMCO GENERAL (Solo Último Año)
# ==========================================
print(f"📂 Procesando {archivo_general}...")
try:
    df_gen = pd.read_csv(archivo_general, encoding='utf-8')
except:
    df_gen = pd.read_csv(archivo_general, encoding='latin-1')

# Corregir columnas
cols_gen_map = {c: 'Año' if 'AÃ±o' in c else 'Cambio de posición' if 'posiciÃ³n' in c else c for c in df_gen.columns}
df_gen.rename(columns=cols_gen_map, inplace=True)

# FILTRO: SOLO EL ÚLTIMO AÑO
ultimo_ano_gen = df_gen['Año'].max()
print(f"   📅 Año seleccionado (General): {ultimo_ano_gen}")

df_gen_final = df_gen[df_gen['Año'] == ultimo_ano_gen].reset_index(drop=True)


# ==========================================
# 2. PROCESAMIENTO: IMCO DESAGREGADO (Comparativa)
# ==========================================
print(f"📂 Procesando {archivo_desagregado}...")
try:
    df_des = pd.read_csv(archivo_desagregado, encoding='utf-8')
except:
    df_des = pd.read_csv(archivo_desagregado, encoding='latin-1')

# Corregir columnas
cols_des_map = {c: 'Subíndice' if 'SubÃ­ndice' in c else c for c in df_des.columns}
df_des.rename(columns=cols_des_map, inplace=True)

# Identificar las 2 últimas fechas disponibles para comparar
fechas_disponibles = sorted(df_des['Date'].unique(), reverse=True)
ultimas_2_fechas = fechas_disponibles[:2]
print(f"   📅 Fechas para comparación (Desagregado): {ultimas_2_fechas}")

# Filtramos solo esos dos años para trabajar
df_des_filtrado = df_des[df_des['Date'].isin(ultimas_2_fechas)].copy()

# GENERAR RANKING (1 al 32) para AMBOS años
# Ascending=False -> El valor más alto es el #1
df_des_filtrado['Ranking'] = df_des_filtrado.groupby(['Date', 'Indicador'])['Value'].rank(ascending=False, method='min')

# SEPARAR Y CALCULAR CAMBIO
fecha_actual = ultimas_2_fechas[0]
fecha_previa = ultimas_2_fechas[1]

# DataFrame del año actual
df_actual = df_des_filtrado[df_des_filtrado['Date'] == fecha_actual].copy()

# DataFrame del año previo (solo columnas clave para el cruce)
df_previo = df_des_filtrado[df_des_filtrado['Date'] == fecha_previa][['Entidad', 'Indicador', 'Ranking']].rename(columns={'Ranking': 'Ranking_Anterior'})

# Unimos la historia al presente
df_final_des = df_actual.merge(df_previo, on=['Entidad', 'Indicador'], how='left')

# CÁLCULO: (Ranking Anterior - Ranking Actual)
# Ejemplo: Antes 10, Hoy 5 -> 10 - 5 = +5 (Mejoró 5 lugares)
df_final_des['Cambio_Posicion'] = df_final_des['Ranking_Anterior'] - df_final_des['Ranking']
df_final_des['Cambio_Posicion'] = df_final_des['Cambio_Posicion'].fillna(0) # Indicadores nuevos sin historia

# LIMPIEZA FINAL
cols_finales = ['Date', 'Entidad', 'Subíndice', 'Indicador', 'Unidad', 'Value', 'Ranking', 'Ranking_Anterior', 'Cambio_Posicion']
df_final_des = df_final_des[cols_finales]


# ==========================================
# 3. RESULTADOS
# ==========================================

print("\n--- IMCO GENERAL (Solo Último Año) ---")
print(df_gen_final.head())

print("\n--- IMCO DESAGREGADO (Con Cambio de Posición Calculado) ---")
# Ejemplo para verificar
print(f"Ejemplo: Aguascalientes (Desagregado)")
print(df_final_des[df_final_des['Entidad'] == 'Aguascalientes'].head())

# Opcional: Guardar
# df_gen_final.to_csv("IMCO_General_Final.csv", index=False)
# df_final_des.to_csv("IMCO_Desagregado_Final_Rankeado.csv", index=False)

📂 Procesando imco_general.csv...
   📅 Año seleccionado (General): 2025-01-01
📂 Procesando imco_desagregado.csv...
   📅 Fechas para comparación (Desagregado): ['2025-01-01', '2024-01-01']

--- IMCO GENERAL (Solo Último Año) ---
          Año              Entidad      Valor Nivel de Competitividad  \
0  2025-01-01     Ciudad de México  75.000000                Muy alta   
1  2025-01-01  Baja California Sur  53.214644                    Alta   
2  2025-01-01           Nuevo León  51.615049                    Alta   
3  2025-01-01              Jalisco  51.128925                    Alta   
4  2025-01-01       Aguascalientes  50.103085              Media alta   

   Ranking  Cambio de posición  
0        1                   0  
1        2                   0  
2        3                   0  
3        4                   3  
4        5                   1  

--- IMCO DESAGREGADO (Con Cambio de Posición Calculado) ---
Ejemplo: Aguascalientes (Desagregado)
         Date         Entidad Subíndi